<a href="https://colab.research.google.com/github/yuto-kobayashi-1/signate-AIQuest/blob/develop/210716_AIQuest_fix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import optuna.integration.lightgbm as lgb_o
import lightgbm as lgb
from copy import deepcopy
import numpy as np
from datetime import datetime , date
from sklearn.inspection import permutation_importance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/signate/competiton/210716_AI Quest/train.csv",encoding="utf_8")
test = pd.read_csv("/content/drive/MyDrive/signate/competiton/210716_AI Quest/test.csv",encoding="utf_8")

In [ ]:
def preprocessing(df):
  df["host_response_rate"] = df["host_response_rate"].str.replace('%', '').astype(float)
  df["host_since"] = (date(2021,7,21) - pd.to_datetime(df["host_since"]).dt.date).dt.days
  # df["first_review"] = (date(2021,7,21) - pd.to_datetime(df["first_review"]).dt.date).dt.days
  # df["last_review"] = (date(2021,7,21) - pd.to_datetime(df["last_review"]).dt.date).dt.days
  # df["review_days"] = (data["first_review"] - data["last_review"])
  df['amenities'] = df['amenities'].str.replace('"', '')
  df['amenities'] = df['amenities'].str.replace('{', '')
  df['amenities'] = df['amenities'].str.replace('}', '')
  
  return df

def label_encorder(df_train,df_test,cols):
  df_train["flg"] = 0
  df_test["y"] = 100
  df_test["flg"] = 1

  df_ = pd.concat([df_train,df_test]).reset_index(drop=True)
  le = LabelEncoder()

  for col in cols:
    df_[col] = df_[col].fillna("OK")
    null_list = list(df_[df_[col] == "OK"].index)
    print(len(null_list))
    encoded = le.fit_transform(df_[col].values)
    decoded = le.inverse_transform(encoded)
    df_[col] = encoded
    df_.loc[null_list,col]=np.nan

  df_train_ = df_[df_["flg"]==0].drop("flg",axis=1)
  df_test_ = df_[df_["flg"]==1].drop(["flg","y"],axis=1)

  return df_train_ ,  df_test_

def get_dummys(df_train,df_test,cols):
  df_train["flg"] = 0
  df_test["y"] = 100
  df_test["flg"] = 1

  df_ = pd.concat([df_train,df_test]).reset_index(drop=True)

  amenities = df_["amenities"].str.get_dummies(sep=',')

  df_ = pd.concat([df_,amenities],axis=1)

  df_train_ = df_[df_["flg"]==0].drop("flg",axis=1)
  df_test_ = df_[df_["flg"]==1].drop(["flg","y"],axis=1)

  return df_train_ ,  df_test_


def train(df,label_cols,drop_cols):
  
  X =  df.drop(drop_cols + ['y'],axis=1)
  y = df['y']

  kf = KFold(n_splits=5,shuffle=True,random_state=0)

  models = []
  for train_index, test_index in kf.split(X, y):
    X_train = X.iloc[train_index]
    y_train =  y.iloc[train_index]
    X_valid = X.iloc[test_index]
    y_valid = y.iloc[test_index]

    X_train, y_train, X_valid, y_valid =  target_encording(X_train, y_train, X_valid, y_valid,label_cols)

  #   params = {
  #   'boosting_type': 'gbdt',
  #   'objective': 'regression',
  #   'metric': 'rmse',
  #   'num_leaves': 64,
  #   'min_data_in_leaf': 20,
  #   'max_depth': 7,
  #   'verbose': 0,
  # }
    params = {'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'boosting_type': 'gbdt',
 'feature_fraction': 0.8160000000000001,
 'feature_pre_filter': False,
 'lambda_l1': 7.849404966982897e-05,
 'lambda_l2': 5.174000510291167e-05,
 'metric': 'rmse',
 'min_child_samples': 50,
 'num_leaves': 44,
 'objective': 'regression'}

    train_data = lgb.Dataset(
      data=X_train, 
      label=y_train,
  )

    validation_data = lgb.Dataset(
      data=X_valid, 
      label=y_valid, 
  )

    model = lgb.train(
      params=params, 
      train_set=train_data, 
      num_boost_round=1000, 
      early_stopping_rounds=100,
      valid_sets=[validation_data],
  )

    models.append(model)

  return models 

def feature_importance(models,data,cols):
  importance = pd.DataFrame(columns=data[cols].columns)

  for model in models:
    imp_ = pd.Series(model.feature_importance(),index=data[cols].columns)
    importance = importance.append(imp_,ignore_index=True)

  importance = importance.mean()

  importance = pd.DataFrame(importance,columns={"importance"}).sort_values("importance",ascending=False)

  importance = pd.DataFrame(importance/importance.sum())
  return importance

def adv_train(train,test,label_cols,drop_cols):

  df_train = train.copy()
  df_test = test.copy()

  df_train = df_train.drop(drop_cols,axis=1)
  df_test = df_test.drop(drop_cols,axis=1)
  
  df_train ,  df_test = label_encorder(df_train,df_test,label_cols)

  df_train = df_train.drop('y',axis=1)

  df_train["flg"] = 0
  df_test["flg"] = 1

  df_ = pd.concat([df_train,df_test]).reset_index(drop=True)

  kf = KFold(n_splits=5,shuffle=True,random_state=0)
  X  = df_ .drop('flg',axis=1)
  y   = df_['flg']

  models = []
  for train_index, test_index in kf.split(X, y):
    X_train = X.iloc[train_index]
    y_train =  y.iloc[train_index]
    X_valid = X.iloc[test_index]
    y_valid = y.iloc[test_index]

    params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 64,
    'min_data_in_leaf': 20,
    'max_depth': 7,
    'verbose': 0,
  }
    train_data = lgb.Dataset(
      data=X_train, 
      label=y_train,
  )

    validation_data = lgb.Dataset(
      data=X_valid, 
      label=y_valid, 
  )

    model = lgb.train(
      params=params, 
      train_set=train_data, 
      num_boost_round=1000, 
      early_stopping_rounds=100,
      valid_sets=[validation_data]
  )

    models.append(model)
  
  preds = np.zeros((len(df_train),len(models)))
  for n , model in enumerate(models):
    # pred = model.predict(df_test[cols],num_iteration=model.best_iteration)
    pred = model.predict(df_train.drop('flg',axis=1),num_iteration=model.best_iteration)
    preds[:,n] = pred
  
  train['pred'] =  np.mean(preds,axis=1)
  train =  train.sort_values('pred',ascending=False)

  return train 

def train_specify(train,valid,label_cols,drop_cols):
  
  X = train.drop(drop_cols+["y"],axis=1)
  y  = train['y']
  X_valid = valid.drop(drop_cols+["y"],axis=1)
  y_valid = valid['y']

  kf = KFold(n_splits=5,shuffle=True,random_state=0)

  models = []
  for train_index, test_index in kf.split(X, y):
    X_train = X.iloc[train_index]
    y_train =  y.iloc[train_index]
    # X_valid = X.iloc[test_index]
    # y_valid = y.iloc[test_index]

    X_train, y_train, X_valid, y_valid =  target_encording(X_train, y_train, X_valid, y_valid,label_cols)

    params = {
    'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'boosting_type': 'gbdt',
 'feature_fraction': 0.8160000000000001,
 'feature_pre_filter': False,
 'lambda_l1': 7.849404966982897e-05,
 'lambda_l2': 5.174000510291167e-05,
 'metric': 'rmse',
 'min_child_samples': 50,
 'num_leaves': 44,
 'objective': 'regression'}

    train_data = lgb.Dataset(
      data=X_train, 
      label=y_train,
  )

    validation_data = lgb.Dataset(
      data=X_valid, 
      label=y_valid, 
  )

    model = lgb.train(
      params=params, 
      train_set=train_data, 
      num_boost_round=1000, 
      early_stopping_rounds=100,
      valid_sets=[validation_data]
  )

    models.append(model)

  return models 

In [ ]:
def param_tuning(train,valid,label_cols,drop_cols):
  
  X_train = train.drop(drop_cols+["y"],axis=1)
  y_train  = train['y']
  X_valid = valid.drop(drop_cols+["y"],axis=1)
  y_valid = valid['y']

  X_train, y_train, X_valid, y_valid =  target_encording(X_train, y_train, X_valid, y_valid,label_cols)

  params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    # 'num_leaves': 64,
    # 'min_data_in_leaf': 20,
    # 'max_depth': 7,
    # 'verbose': 0,
  }
  train_data = lgb_o.Dataset(
      data=X_train, 
      label=y_train,
  )

  validation_data = lgb_o.Dataset(
      data=X_valid, 
      label=y_valid, 
  )

  model = lgb_o.train(
      params=params, 
      train_set=train_data, 
      num_boost_round=1000, 
      early_stopping_rounds=100,
      valid_sets=[validation_data]
  )

  return model

In [ ]:
def train_permutation(train,valid,label_cols,select_cols):
  
  X = train[select_cols]
  y  = train['y']
  X_valid = valid[select_cols]
  y_valid = valid['y']

  kf = KFold(n_splits=5,shuffle=True,random_state=0)

  models = []
  for train_index, test_index in kf.split(X, y):
    X_train = X.iloc[train_index]
    y_train =  y.iloc[train_index]
    # X_valid = X.iloc[test_index]
    # y_valid = y.iloc[test_index]

    X_train, y_train, X_valid, y_valid =  target_encording(X_train, y_train, X_valid, y_valid,label_cols)

    params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 64,
    'min_data_in_leaf': 20,
    'max_depth': 7,
    'verbose': 0,
  }
    train_data = lgb.Dataset(
      data=X_train, 
      label=y_train,
  )

    validation_data = lgb.Dataset(
      data=X_valid, 
      label=y_valid, 
  )

    model = lgb.train(
      params=params, 
      train_set=train_data, 
      num_boost_round=1000, 
      early_stopping_rounds=100,
      valid_sets=[validation_data]
  )

    models.append(model)

  return models 

def train_permutation_useValid(train,valid,label_cols,select_cols):
  
  data = pd.concat([train,valid])

  X = data[select_cols]
  y  = data['y']

  kf = KFold(n_splits=5,shuffle=True,random_state=0)

  models = []
  for train_index, test_index in kf.split(X, y):
    X_train = X.iloc[train_index]
    y_train =  y.iloc[train_index]
    X_valid = X.iloc[test_index]
    y_valid = y.iloc[test_index]

    X_train, y_train, X_valid, y_valid =  target_encording(X_train, y_train, X_valid, y_valid,label_cols)

    params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 64,
    'min_data_in_leaf': 20,
    'max_depth': 7,
    'verbose': 0,
  }
    train_data = lgb.Dataset(
      data=X_train, 
      label=y_train,
  )

    validation_data = lgb.Dataset(
      data=X_valid, 
      label=y_valid, 
  )

    model = lgb.train(
      params=params, 
      train_set=train_data, 
      num_boost_round=1000, 
      early_stopping_rounds=100,
      valid_sets=[validation_data]
  )

    models.append(model)

  return models 

In [ ]:
def target_encording(tr_x, tr_y, va_x, va_y, cols):

  for c in cols:

    data_tmp = pd.DataFrame({c:tr_x[c],'target':tr_y})
    target_mean = data_tmp.groupby(c)['target'].mean()
    #バリデーションデータのカテゴリを置換
    va_x.loc[:,c] = va_x[c].map(target_mean)

    #学習データの変換後の値を格納する配列を準備
    tmp = np.repeat(np.nan,tr_x.shape[0])
    kf_encording = KFold(n_splits=5,shuffle=True,random_state=72)
    for idx_1, idx_2 in kf_encording.split(tr_x):
      #out-of-foldで各カテゴリにおける目的変数の平均を計算
      target_mean = data_tmp.iloc[idx_1].groupby(c)['target'].mean()
      #変換後の値を一時配列に格納
      tmp[idx_2] = tr_x[c].iloc[idx_2].map(target_mean)

    tr_x.loc[:,c] = tmp

  return tr_x, tr_y, va_x, va_y

def target_encording_test(tr_x, tr_y, va_x, cols):
  for c in cols:
    data_tmp = pd.DataFrame({c:tr_x[c],'target':tr_y})
    target_mean = data_tmp.groupby(c)['target'].mean()
    #バリデーションデータのカテゴリを置換
    va_x.loc[:,c] = va_x[c].map(target_mean)

  return va_x

def predict(models,df):
  preds = np.zeros((len(df),len(models)))
  for n , model in enumerate(models):
    # pred = model.predict(df_test[cols],num_iteration=model.best_iteration)
    pred = model.predict(df.drop("id",axis=1),num_iteration=model.best_iteration)
    preds[:,n] = pred

  score = pd.DataFrame()
  score["id"] = df["id"].astype(int)
  score["pred"] = np.mean(preds,axis=1)
  
  return score

In [ ]:
from sklearn.metrics import mean_squared_error, make_scorer
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred)) 
mse_scorer = make_scorer(rmse)

In [ ]:
def permutation_train(train,valid,label_cols,drop_cols):
  
  X = train.drop(drop_cols+["y"],axis=1)
  y  = train['y']
  X_valid = valid.drop(drop_cols+["y"],axis=1)
  y_valid = valid['y']

  kf = KFold(n_splits=5,shuffle=True,random_state=0)

  models = []
  perm_imp_list = []
  for train_index, test_index in kf.split(X, y):
    X_train = X.iloc[train_index]
    y_train =  y.iloc[train_index]
    # X_valid = X.iloc[test_index]
    # y_valid = y.iloc[test_index]

    X_train, y_train, X_valid, y_valid =  target_encording(X_train, y_train, X_valid, y_valid,label_cols)

    parms= {'X': X_train.values,
                  'y': y_train.values,
                  'eval_set': (X_valid.values, y_valid.values),
                  'early_stopping_rounds': 100,
                  'eval_metric':'rmse',
                  'verbose': False,}

    model = lgb.LGBMRegressor(objective="regression", n_estimators=100, importance_type="gain", num_leave=64,max_depth=7)

    # gbm = model.fit(**parms, callbacks=callbacks)
    model = model.fit(**parms)

    models += [model]

    # run permutation importance
    result = permutation_importance(model, X_valid.values, y_valid.values, scoring=mse_scorer, n_repeats=10, n_jobs=-1, random_state=71)
    perm_imp_df = pd.DataFrame({"importances_mean":result["importances_mean"], "importances_std":result["importances_std"]}, index=X.columns)
    perm_imp_list += [perm_imp_df]

  return perm_imp_list 

In [ ]:
len(data)

55583

In [ ]:
len(test)

18528

In [ ]:
cols = ['accommodates','bathrooms', 'bed_type', 'bedrooms',
       'beds', 'cancellation_policy', 'city', 'cleaning_fee',
       'host_has_profile_pic', 'host_identity_verified',
       'instant_bookable',
       'latitude', 'longitude', 'neighbourhood', 'number_of_reviews',
       'property_type', 'review_scores_rating', 'room_type',"host_response_rate",
        "host_since","amenities"
        ]

drop_cols = ["id","amenities","description","first_review","last_review","name","thumbnail_url","zipcode"]

label_cols = ['bed_type','cancellation_policy',"city","cleaning_fee","host_has_profile_pic","host_identity_verified","instant_bookable","neighbourhood","property_type","room_type"]


In [ ]:
data = preprocessing(data)
test = preprocessing(test)

In [ ]:
df_train , df_test  = get_dummys(data,test,label_cols)
# df_train , df_test  = data,test

adversarial train を行いtestデータに似ているtrainデータをvalidデータにする

In [ ]:
# df_avd_train = adv_train(df_train , df_test,label_cols,drop_cols)

In [ ]:
# print(len(df_train)*0.2)

In [ ]:
# train_data = df_avd_train[11116:]
# valid_data = df_avd_train[:11116]

In [ ]:
# train_data

In [ ]:
# train_data.to_csv("/content/drive/MyDrive/signate/competiton/210716_AI Quest/train_data.csv",header=True,index=False)
# valid_data.to_csv("/content/drive/MyDrive/signate/competiton/210716_AI Quest/valid_data.csv",header=True,index=False)
# df_test.to_csv("/content/drive/MyDrive/signate/competiton/210716_AI Quest/test_data.csv",header=True,index=False)

In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/signate/competiton/210716_AI Quest/train_data.csv")
valid_data = pd.read_csv("/content/drive/MyDrive/signate/competiton/210716_AI Quest/valid_data.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.



In [ ]:
train_data = train_data.drop('pred',axis=1)
valid_data = valid_data.drop('pred',axis=1)

permutation_importance

In [ ]:
# imoprtanceDF = permutation_train(train_data,valid_data,label_cols,drop_cols)

特徴量選択

In [ ]:
# imoprtanceDF_mean = pd.concat([imoprtanceDF[i][['importances_mean']] for i in range(len(imoprtanceDF))],axis=1)
# imoprtanceDF_mean = pd.DataFrame(imoprtanceDF_mean.mean(axis=1),columns={'importances_mean'}).sort_values('importances_mean')
# imoprtanceDF_mean = imoprtanceDF_mean[imoprtanceDF_mean['importances_mean']<0]

In [ ]:
imoprtanceDF_mean

,importances_mean
bathrooms,-21.222306
bedrooms,-20.958377
accommodates,-14.974927
longitude,-8.126308
number_of_reviews,-3.229491
...,...
Baby bath,-0.000308
Smartlock,-0.000140
Window guards,-0.000131
Private living room,-0.000072


In [ ]:
features = list(imoprtanceDF_mean.reset_index()['index'].values)
print(features)

['bathrooms', 'bedrooms', 'accommodates', 'longitude', 'number_of_reviews', 'host_response_rate', 'latitude', 'review_scores_rating', 'neighbourhood', 'room_type', 'Cable TV', 'Pool', 'city', 'Family/kid friendly', 'TV', 'Hot tub', 'Buzzer/wireless intercom', 'Suitable for events', 'Gym', 'host_since', 'property_type', 'Indoor fireplace', 'cleaning_fee', 'Lock on bedroom door', 'Elevator', 'Essentials', 'translation missing: en.hosting_amenity_50', 'Breakfast', 'Doorman', 'Free parking on premises', 'Elevator in building', 'Internet', 'Laptop friendly workspace', 'Self Check-In', 'Wireless Internet', 'Fire extinguisher', 'Carbon monoxide detector', 'Smoke detector', 'Washer', 'First aid kit', 'cancellation_policy', 'Private entrance', 'Air conditioning', 'Cat(s)', 'Iron', 'Shampoo', 'Pets allowed', 'Children’s books and toys', 'host_identity_verified', 'Crib', 'Dishwasher', 'Long term stays allowed', 'translation missing: en.hosting_amenity_49', 'Pets live on this property', '24-hour c

In [ ]:
per_label_cols = ['bed_type','cancellation_policy',"city","cleaning_fee","host_identity_verified","instant_bookable","neighbourhood","property_type","room_type"]



```
# これはコードとして書式設定されます
```

学習

In [ ]:
# param_model = param_tuning(train_data,valid_data,label_cols,drop_cols)

In [ ]:
param_model.params

{'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'boosting_type': 'gbdt',
 'feature_fraction': 0.8160000000000001,
 'feature_pre_filter': False,
 'lambda_l1': 7.849404966982897e-05,
 'lambda_l2': 5.174000510291167e-05,
 'metric': 'rmse',
 'min_child_samples': 50,
 'num_leaves': 44,
 'objective': 'regression'}

In [ ]:
# models = train(df_train,label_cols,drop_cols)
models = train_specify(train_data,valid_data,label_cols,drop_cols)
# models = train_permutation(train_data,valid_data,per_label_cols,features)
# models = train_permutation_useValid(train_data,valid_data,per_label_cols,features)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[1]	valid_0's rmse: 175.967
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 168.643
[3]	valid_0's rmse: 163.127
[4]	valid_0's rmse: 156.97
[5]	valid_0's rmse: 151.331
[6]	valid_0's rmse: 146.69
[7]	valid_0's rmse: 142.416
[8]	valid_0's rmse: 139.009
[9]	valid_0's rmse: 136.064
[10]	valid_0's rmse: 133.438
[11]	valid_0's rmse: 131.669
[12]	valid_0's rmse: 129.713
[13]	valid_0's rmse: 128.04
[14]	valid_0's rmse: 126.45
[15]	valid_0's rmse: 125.133
[16]	valid_0's rmse: 123.848
[17]	valid_0's rmse: 122.904
[18]	valid_0's rmse: 121.962
[19]	valid_0's rmse: 121.331
[20]	valid_0's rmse: 120.683
[21]	valid_0's rmse: 120.083
[22]	valid_0's rmse: 119.619
[23]	valid_0's rmse: 119.06
[24]	valid_0's rmse: 118.581
[25]	valid_0's rmse: 118.128
[26]	valid_0's rmse: 117.715
[27]	valid_0's rmse: 117.418
[28]	valid_0's rmse: 117.174
[29]	valid_0's rmse: 116.843
[30]	valid_0's rmse: 116.738
[31]	valid_0's rmse: 116.396
[32]	valid_0's rmse: 116.204
[33]	valid_0's rmse: 11

In [ ]:
# non turning : 104.17107190259549
# turning : 103.27262504686973
# perrmutation & adv : 133.03601307479352
# adv :131.6204179616452
# adv + tuning :128.60321638661898
# perrmutation : 104.77516543443774
aucs=[]
for model in models:
  aucs.append(model.best_score["valid_0"]["rmse"])
np.mean(aucs)

128.60321638661898

In [ ]:
# importance = feature_importance(models,df_train,df_train.drop(drop_cols+["y"],axis=1).columns)
importance = feature_importance(models,train_data,train_data.drop(drop_cols+["y"],axis=1).columns)
importance[importance["importance"]>0].head(50)
# importance = feature_importance(models,df_train,features)
# importance[importance["importance"]>0]

,importance
neighbourhood,0.105469
longitude,0.080960
latitude,0.067686
host_since,0.064330
accommodates,0.061357
bathrooms,0.054773
number_of_reviews,0.048805
review_scores_rating,0.048041
property_type,0.041860
host_response_rate,0.039163


In [ ]:
df_test = target_encording_test(df_train.drop(drop_cols+["y"],axis=1),df_train['y'],df_test,label_cols)

In [ ]:
drop_cols.remove("id")
score = predict(models,df_test.drop(drop_cols,axis=1))

In [ ]:
score

,id,pred
55583,0,273.786994
55584,1,131.785689
55585,2,106.578481
55586,3,128.382493
55587,4,135.871446
...,...,...
74106,18523,187.569360
74107,18524,139.334029
74108,18525,182.170686
74109,18526,121.079364


In [ ]:
score.dtypes

id        int64
pred    float64
dtype: object

In [ ]:
score.to_csv("/content/drive/MyDrive/signate/competiton/210716_AI Quest/submit.csv",header=False,index=False)